In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.datasets as ds

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense,Input
from tensorflow.keras.optimizers import Adam

In [2]:
(x_train,y_train),(x_test,y_test)=ds.mnist.load_data()
x_train.shape

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


(60000, 28, 28)

In [3]:
x_train=x_train.reshape(60000,28,28,1)
x_test=x_test.reshape(10000,28,28,1)
x_train.shape

(60000, 28, 28, 1)

In [6]:
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0

In [7]:
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

In [8]:
cnn=Sequential()
input = Input(shape=(28,28,1))
cnn.add(input)
cnn.add(Conv2D(6,(5,5),padding='same',activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2),strides=2))
cnn.add(Conv2D(16,(5,5),padding='valid',activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2),strides=2))
cnn.add(Conv2D(120,(5,5),padding='valid',activation='relu'))
cnn.add(Flatten())
cnn.add(Dense(units=84,activation='relu'))
cnn.add(Dense(units=10,activation='softmax'))

In [9]:
cnn.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])
cnn.fit(x_train,y_train,batch_size=128,epochs=30,validation_data=(x_test,y_test),verbose=2)

Epoch 1/30
469/469 - 42s - 89ms/step - accuracy: 0.9075 - loss: 0.3163 - val_accuracy: 0.9717 - val_loss: 0.0899
Epoch 2/30
469/469 - 41s - 86ms/step - accuracy: 0.9731 - loss: 0.0864 - val_accuracy: 0.9781 - val_loss: 0.0676
Epoch 3/30
469/469 - 38s - 82ms/step - accuracy: 0.9821 - loss: 0.0590 - val_accuracy: 0.9858 - val_loss: 0.0447
Epoch 4/30
469/469 - 39s - 82ms/step - accuracy: 0.9847 - loss: 0.0469 - val_accuracy: 0.9860 - val_loss: 0.0434
Epoch 5/30
469/469 - 37s - 79ms/step - accuracy: 0.9880 - loss: 0.0372 - val_accuracy: 0.9881 - val_loss: 0.0356
Epoch 6/30
469/469 - 42s - 89ms/step - accuracy: 0.9896 - loss: 0.0320 - val_accuracy: 0.9880 - val_loss: 0.0360
Epoch 7/30
469/469 - 39s - 83ms/step - accuracy: 0.9909 - loss: 0.0279 - val_accuracy: 0.9887 - val_loss: 0.0353
Epoch 8/30
469/469 - 40s - 85ms/step - accuracy: 0.9924 - loss: 0.0243 - val_accuracy: 0.9901 - val_loss: 0.0340
Epoch 9/30
469/469 - 42s - 89ms/step - accuracy: 0.9929 - loss: 0.0211 - val_accuracy: 0.9914 - 

In [10]:
res=cnn.evaluate(x_test,y_test,verbose=0)
print('정확도=',res[1]*100)

정확도= 98.94000291824341


In [11]:
cnn.save('cnn_v2.h5')

저장한 모델 불러오기

LeNet 재현 실습을 통해 저장한 모델 불러오기
MNIST 데이터셋으로 학습된 손글씨 숫자 인식 모델

In [12]:
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt

In [13]:
model = tf.keras.models.load_model('cnn_v2.h5')

In [14]:
def reset():
    global img
    img = np.ones((200, 520, 3), dtype=np.uint8) * 255
    for i in range(5):
        cv.rectangle(img, (10 + i * 100, 50), (10 + (i + 1) * 100, 150), (0, 0, 255))
    cv.putText(img, 'e:erase s:show r:recognition q:quit', (10, 40), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1)

In [15]:
def grab_numerals():
    numerals = []
    for i in range(5):
        roi = img[51:149, 11 + i * 100:9 + (i + 1) * 100, 0]
        roi = 255 - cv.resize(roi, (28, 28), interpolation=cv.INTER_CUBIC)
        numerals.append(roi)
    numerals = np.array(numerals)
    return numerals

In [16]:
def show():
    numerals = grab_numerals()
    plt.figure(figsize=(25, 5))
    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(numerals[i], cmap='gray')
        plt.xticks([]); plt.yticks([])
    plt.show()

In [17]:
def recognition():
    numerals = grab_numerals()
    numerals = numerals.reshape(5, 28, 28) # 지난 실습 7-7.ipynb (5, 784)에서 (5, 28, 28) 로 변경
    numerals = numerals.astype(np.float32) / 255.0
    res = model.predict(numerals)  # 신경망 모델로 예측
    class_id = np.argmax(res, axis=1)
    for i in range(5):
        cv.putText(img, str(class_id[i]), (50 + i * 100, 180), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)
    # winsound.Beep(1000, 500)

In [18]:
BrushSiz = 4
LColor = (0, 0, 0)

def writing(event, x, y, flags, param):
    if event == cv.EVENT_LBUTTONDOWN:
        cv.circle(img, (x, y), BrushSiz, LColor, -1)
    elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_LBUTTON:
        cv.circle(img, (x, y), BrushSiz, LColor, -1)

In [ ]:
reset()
cv.namedWindow('Writing')
cv.setMouseCallback('Writing', writing)

while True:
    cv.imshow('Writing', img)
    key = cv.waitKey(1)
    if key == ord('e'):
        reset()
    elif key == ord('s'):
        show()
    elif key == ord('r'):
        recognition()
    elif key == ord('q'):
        break

cv.destroyAllWindows()